In [1]:
!pip install pycrypto

     |████████████████████████████████| 446 kB 5.4 MB/s 
  Created wheel for pycrypto: filename=pycrypto-2.6.1-cp37-cp37m-linux_x86_64.whl size=499932 sha256=65b088edcb6396c8245d7024d7155dc145199334eebd3f20aa6a04c225079216
  Stored in directory: /root/.cache/pip/wheels/cf/85/ba/bbd7c96add459de7598fb424e5ff2309baf2095c844ac0f191
Successfully built pycrypto


In [2]:
from Crypto.Cipher import DES

In [3]:
def hex2string(hex_message):
    '''
    >>> hex2string('61')
    'a'
    >>> hex2string('776f726c64')
    'world'
    >>> hex2string('68656c6c6f')
    'hello'
    '''
    ret = ""
    for i in range(0,len(hex_message),2):
        ret += chr(int(hex_message[i:i+2],16))

    return ret

In [4]:
def bin2hex(binary):

    return hex(int(binary, 2))

def string2hex(message):
    ret = ""
    for c in message:
        ret += hex(ord(c))[2:].rjust(2,'0')
    return ret

def hex2bin(hex_value):
    return bin(int(hex_value, 16))[2:].zfill(8)

In [5]:
key_shifts = [1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1]

PC1 = [57, 49, 41, 33, 25, 17, 9, 1, 58, 50, 42, 34, 26, 18, 10, 2, 59, 51, 43, 35, 27, 19, 11, 3, 60, 52, 44, 36, 63,
       55, 47, 39, 31, 23, 15, 7, 62, 54, 46, 38, 30, 22, 14, 6, 61, 53, 45, 37, 29, 21, 13, 5, 28, 20, 12, 4]

PC2 = [14, 17, 11, 24, 1, 5, 3, 28, 15, 6, 21, 10, 23, 19, 12, 4, 26, 8, 16, 7, 27, 20, 13, 2, 41, 52, 31, 37, 47, 55,
       30, 40, 51, 45, 33, 48, 44, 49, 39, 56, 34, 53, 46, 42, 50, 36, 29, 32]

IP = [58, 50, 42, 34, 26, 18, 10, 2,
      60, 52, 44, 36, 28, 20, 12, 4,
      62, 54, 46, 38, 30, 22, 14, 6,
      64, 56, 48, 40, 32, 24, 16, 8,
      57, 49, 41, 33, 25, 17, 9, 1,
      59, 51, 43, 35, 27, 19, 11, 3,
      61, 53, 45, 37, 29, 21, 13, 5,
      63, 55, 47, 39, 31, 23, 15, 7]

E = [32, 1, 2, 3, 4, 5, 4, 5, 6, 7, 8, 9, 8, 9, 10, 11, 12, 13, 12, 13, 14, 15, 16, 17, 16, 17, 18, 19, 20, 21, 20, 21,
     22, 23, 24, 25, 24, 25, 26, 27, 28, 29, 28, 29, 30, 31, 32, 1]
IP_inverse = [40, 8, 48, 16, 56, 24, 64, 32,
              39, 7, 47, 15, 55, 23, 63, 31,
              38, 6, 46, 14, 54, 22, 62, 30,
              37, 5, 45, 13, 53, 21, 61, 29,
              36, 4, 44, 12, 52, 20, 60, 28,
              35, 3, 43, 11, 51, 19, 59, 27,
              34, 2, 42, 10, 50, 18, 58, 26,
              33, 1, 41, 9, 49, 17, 57, 25]
S = \
[
    [
        [14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
        [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
        [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
        [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]
    ],
    [
        [15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
        [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
        [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
        [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]
    ],
    [
        [10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
        [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
        [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
        [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]
    ],
    [
        [7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
        [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
        [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
        [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]
    ],
    [
        [2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
        [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
        [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
        [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]
    ],
    [
        [12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
        [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
        [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
        [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]
    ],
    [
        [4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
        [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
        [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
        [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12]
    ],
    [
        [13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
        [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
        [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
        [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]
    ]
]

P = [
    16, 7, 20, 21, 29, 12, 28, 17,
    1, 15, 23, 26, 5, 18, 31, 10,
    2, 8, 24, 14, 32, 27, 3, 9,
    19, 13, 30, 6, 22, 11, 4, 25]

In [15]:
def left_shift(a,steps):
    a1=a[steps:]
    a2=a[:steps]
    result = a1 + a2
    return result

In [6]:
def bytes2binary(arr):

    temp=str(hex2bin(arr.hex()))

    answer =  temp
    return answer

In [ ]:
print('bytes to binary')
print(bytes2binary(b'\x01'))
print(bytes2binary(b'\x03'))
print(bytes2binary(b'\xf0'))
print(bytes2binary(b'\xf0\x80'))

bytes to binary
00000001
00000011
11110000
1111000010000000


In [7]:
def binary2bytes(string): #took help from stackoverflow
    # https://stackoverflow.com/questions/32675679/convert-binary-string-to-bytearray-in-python-3

    integer = int(string, 2)
    b = bytearray()
    while integer:
        b.append(integer & 0xff)
        integer >>= 8
    return bytes(b[::-1])

print('bytes to binary')
print(binary2bytes('00000001'))
print(binary2bytes('00000011'))
print(binary2bytes('11110000'))
print(binary2bytes('1111000010000000'))

bytes to binary
b'\x01'
b'\x03'
b'\xf0'
b'\xf0\x80'


In [8]:
def bin_xor(a, b):

    astring = a.zfill(max(len(a),len(b)))
    bstring = b.zfill(max(len(a),len(b)))
    answer = ''
    for i in range(len(astring)):
        part1 = astring[i]
        part2 = bstring[i]
        if part1 == part2:
            answer =answer + "0"
        else:
            answer = answer +  "1"
    return answer

print("binary xor")
print(bin_xor('1011','0000'))
print(bin_xor('1','0000'))
print(bin_xor('1101','1011'))
print(bin_xor('10101010','01010101'))

binary xor
1011
0001
0110
11111111


In [16]:
print('DES SUBkeys')
def des_subkeys_creation(K):
    temporarykey = ''
    for z in K:
        while len(temporarykey) < len(PC1):
            for iter1 in PC1:
                for iter2 in range(64):
                    if iter1 == iter2:
                        temporarykey =temporarykey + K[iter1-1]
    C_key = temporarykey[0:28]
    D_key = temporarykey[28:57]
    answer_keys = []
    for i1 in range(len(key_shifts)):
        C_key = left_shift(C_key, key_shifts[i1])
        D_key = left_shift(D_key, key_shifts[i1])
        concatenated_string = C_key + D_key
        final_s = ''
        while len(final_s) < len(PC2):
            for iter1 in PC2:
                for iter2 in range(57):
                    if iter2 == iter1:
                        final_s = final_s + concatenated_string[iter2 - 1]
            answer_keys.append(final_s)
    return answer_keys
sub_keys=des_subkeys_creation('0001001100110100010101110111100110011011101111001101111111110001')

DES SUBkeys


In [11]:
def messagepermutation(m,typearray):
    temporarykey = ''
    for z in m:
        while len(temporarykey) < len(typearray):
            for iter1 in typearray:
                for iter2 in range(len(typearray)+1):
                    if iter1 == iter2:
                        temporarykey = temporarykey + m[iter1 - 1]
        return temporarykey

message = messagepermutation('0000000100100011010001010110011110001001101010111100110111101111',IP)
l_message = message[0:32]
r_message = message[32:65]

In [10]:
def f_function(r0, key):
    iter1 = 0
    list1 = []
    list = []
    f_str = ''
    r_out = messagepermutation(r0, E)
    xored = bin_xor(r_out, key)
    interim=''
    interm1 =''
    while iter1 < len(xored):
        list1.append(xored[iter1:iter1 + 6])
        iter1 = iter1 + 6
    for iter2 in range(len(list1)):
        list_bit = list1[iter2]
        S_output = S[iter2]
        interm =(list_bit[0] + list_bit[-1])
        row_S = int(interm, 2)
        interim1 = list_bit[1:5]
        column_S = int(interim1, 2)
        t1 = S_output[row_S]
        t2 = t1[column_S]
        list.append(bin(t2)[2:].zfill(4))
    for iter3 in range(len(list)):
        f_str = f_str + list[iter3]
    f_perm_string = messagepermutation(f_str, P)
    return f_perm_string

print('F function')

F function


In [13]:
print('encryption')
key= b'\x13\x34\x57\x79\x9b\xbc\xdf\xf1'
message = b'\x01\x23\x45\x67\x89\xab\xcd\xef'

encryption


In [17]:
def encryption_des_function(key,message):
    l_message=''
    r_message = ''
    bin_key = bytes2binary(key)
    new_bin_key = bin_key.zfill(64)
    bin_message = bytes2binary(message)
    new_bin_message = bin_message.zfill(64)
    des_subkeys = des_subkeys_creation(new_bin_key)
    permutated_message = messagepermutation(new_bin_message,IP)
    l_message = permutated_message[0:32]
    r_message = permutated_message[32:65]
    temp_r_message = ''
    final=''
    for i in range(16):
        r_message1 = f_function(r_message,des_subkeys[i])
        l_message = bin_xor(l_message,r_message1)
        temp_r_message = r_message
        r_message = l_message
        l_message = temp_r_message

    final = r_message+l_message
    # print(l_message)
    # print(r_message)
    ip_cipher_message = messagepermutation(final,IP_inverse)
    ip_cipher_message1 = binary2bytes(ip_cipher_message)

    return ip_cipher_message1



print(encryption_des_function(key,message))

b'\x85\xe8\x13T\x0f\n\xb4\x05'


In [20]:
import os
message = os.urandom(8)
key = os.urandom(8)
print(message)
print(key)

b'\xa1\xe6\x9fZ\xdf\xde\xc7\xa8'
b'\x1dS\x8c\xd4\xf3X\xd9\xfc'


In [19]:
#key= b'\x13\x34\x57\x79\x9b\xbc\xdf\xf1'
#message = b'\x01\x23\x45\x67\x89\xab\xcd\xef'
def des_library_encryption(message,key):
  cipher = DES.new(key, DES.MODE_ECB)
  plaintext = message
  ciphermsg = cipher.encrypt(plaintext)
  return ciphermsg
#print(des_library_encryption(message,key))
import os
def are_random_all_pass(number_of_iteration):
  result_list = []
  for i in range(number_of_iteration):
    message =  os.urandom(8)
    key = os.urandom(8)
    my_encryption = encryption_des_function(key,message)
    library_encryption = des_library_encryption(message,key)
    #print('Ciphertext from my encryption:',my_encryption)
    #print('Ciphertext from Cipher.text library:',library_encryption)
    if my_encryption == library_encryption:
      print(True)
    else:
      print(False)
are_random_all_pass(100)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
